# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys
sys.path.append("..")

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_to_load = "weather_data.csv"
weather_data = pd.read_csv(weather_data_to_load)

weather_data.head()

,Unnamed: 0,City,Country,Longitude,Latitude,Temperature,Humidity,Cloudiness,Windspeed
0,0,tuktoyaktuk,CA,-133.0374,69.4541,30.20,93,100,47.18
1,1,huarmey,PE,-78.1522,-10.0681,72.05,74,11,9.51
2,2,bubaque,GW,-15.8333,11.2833,78.57,62,10,9.26
3,3,ancud,CL,-73.8203,-41.8697,58.42,77,43,12.91
4,4,tessalit,ML,1.0114,20.1986,83.75,14,4,6.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = weather_data[["Latitude", "Longitude"]]
humidity = weather_data["Humidity"].astype(float)

In [4]:
fig1 = gmaps.figure(zoom_level=2, center=[0,0], layout={'width': '700px', 'height': '500px'})
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=100, point_radius=3, dissipating=False)
fig1.add_layer(heat_layer)
fig1

Figure(layout=FigureLayout(height='500px', width='700px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
# Temp between 20-65
# Windspeed less than 15
# Cloudiness less than 50
# Cities in the US

prefer_df = weather_data.loc[(weather_data["Temperature"] >= 20) & (weather_data["Temperature"] <= 65) & 
                            (weather_data["Windspeed"] < 15) & (weather_data["Cloudiness"] <50) 
                             & (weather_data["Country"] == 'US')] 

print(len(prefer_df))
# prefer_df = prefer_df.rename(columns={'Longitude': 'Lng', 'Latitude': 'Lat'})

prefer_df

12


,Unnamed: 0,City,Country,Longitude,Latitude,Temperature,Humidity,Cloudiness,Windspeed
42,42,winslow,US,-110.6974,35.0242,44.78,45,0,10.36
195,195,fallon,US,-118.7774,39.4735,26.47,59,40,3.44
242,242,lamar,US,-95.5836,33.6668,48.96,53,0,14.97
286,286,saint george,US,-113.5841,37.1041,45.19,57,40,0.00
315,315,lincoln,US,-96.6670,40.8000,23.83,59,0,8.05
407,407,bremerton,US,-122.6326,47.5673,36.09,94,0,5.75
414,414,ketchikan,US,-131.6461,55.3422,27.07,59,0,9.22
437,437,north bend,US,-124.2243,43.4065,42.44,80,0,8.05
474,474,carlyle,US,-89.3726,38.6103,34.21,61,8,5.99
493,493,sitka,US,-135.3300,57.0531,26.49,80,20,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [47]:
hotel_df= prefer_df.rename(columns={'Longitude': 'Lng', 'Latitude': 'Lat'})
hotel_df["Hotel Name"] = ''

hotel_df

,Unnamed: 0,City,Country,Lng,Lat,Temperature,Humidity,Cloudiness,Windspeed,Hotel Name
42,42,winslow,US,-110.6974,35.0242,44.78,45,0,10.36,
195,195,fallon,US,-118.7774,39.4735,26.47,59,40,3.44,
242,242,lamar,US,-95.5836,33.6668,48.96,53,0,14.97,
286,286,saint george,US,-113.5841,37.1041,45.19,57,40,0.00,
315,315,lincoln,US,-96.6670,40.8000,23.83,59,0,8.05,
407,407,bremerton,US,-122.6326,47.5673,36.09,94,0,5.75,
414,414,ketchikan,US,-131.6461,55.3422,27.07,59,0,9.22,
437,437,north bend,US,-124.2243,43.4065,42.44,80,0,8.05,
474,474,carlyle,US,-89.3726,38.6103,34.21,61,8,5.99,
493,493,sitka,US,-135.3300,57.0531,26.49,80,20,4.61,


In [60]:
# Set parameters to search for a hotel
&locations = hotel_df[["Lat", "Lng"]]
target_search = "Hotel"
# radius = 10000
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for loc in locations:
    
    my_params  = {
        "location": loc, # pass a single location only
        "keyword": target_search,
        "radius": radius,
        "key": g_key
    }
    
    first_hotels = requests.get(base_url,params = my_params).json()
    
    try:
        print(first_hotels["results"][0]["name"])
        hotel_df.loc[index, "Hotel Name"]=first_hotels["results"][0]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"




# radius = 10000
# params = {
#     "categories":'accommodation.hotel',
#     "apiKey": g_key,
#     "limit":20
# }


# for index, row in hotel_df.iterrows():
#     lat = row["Lat"]
#     lng = row["Lng"]
#     params["filter"] = f"circle:{lng},{lat},{radius}"    
#     params["bias"] = f"proximity:{lng},{lat}"
    
#     # Set base URL
#     base_url = "https://api.geoapify.com/v2/places"
#     hotel_name = requests.get(base_url, params=params)
    
#     hotel_name = hotel_name.json()
    
#     try:
#         hotel_df.loc[index, "Hotel Name"] = hotel_name["features"][0]["properties"]["name"]
#     except (KeyError, IndexError):
#         # If no hotel is found, set the hotel name as "No hotel found".
#         hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
#     # Log the search results
#     print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# print('--------- Finished Hotel Search ---------------')

# Display sample data
hotel_df
    

# locations = hotel_df[["Lat", "Lng"]]
# humidity = hotel_df["Humidity"].astype(float)
# fig2 = gmaps.figure(zoom_level=2, center=[0,0], layout={'width': '700px', 'height': '500px'})
# heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=100, point_radius=3, dissipating=False)
# fig2.add_layer(heat_layer)
# fig2

,Unnamed: 0,City,Country,Lng,Lat,Temperature,Humidity,Cloudiness,Windspeed,Hotel Name
42,42,winslow,US,-110.6974,35.0242,44.78,45,0,10.36,No hotel found
195,195,fallon,US,-118.7774,39.4735,26.47,59,40,3.44,No hotel found
242,242,lamar,US,-95.5836,33.6668,48.96,53,0,14.97,No hotel found
286,286,saint george,US,-113.5841,37.1041,45.19,57,40,0.00,No hotel found
315,315,lincoln,US,-96.6670,40.8000,23.83,59,0,8.05,No hotel found
407,407,bremerton,US,-122.6326,47.5673,36.09,94,0,5.75,No hotel found
414,414,ketchikan,US,-131.6461,55.3422,27.07,59,0,9.22,No hotel found
437,437,north bend,US,-124.2243,43.4065,42.44,80,0,8.05,No hotel found
474,474,carlyle,US,-89.3726,38.6103,34.21,61,8,5.99,No hotel found
493,493,sitka,US,-135.3300,57.0531,26.49,80,20,4.61,No hotel found


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
